In [1]:
# Do not delete this cell. It ensures that you can do the imports,
# load datasets etc. in the same fashion as in any Python script
# in the project template.


import sys
sys.path.insert(0, '../..')
from bld.project_paths import project_paths_join as ppj
from bld.project_paths import project_paths as pp



In [2]:
import numpy as np
import json
import pickle
import pandas as pd
from scipy.stats import sem
import seaborn as sns
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

from matplotlib import rc
rc('text', usetex=True)
plt.rcParams.update({'font.size': 20})


# Are fully algortihmic markets more collusive than fully human markets?

To answer this question we will compare the outcomes for the algorithmic market to the outcomes in the last super game of the human markets. We focus on the last super game as this allowed humans for some learning. Arguably this is the most accurate comparision.

First we load the needed data.

In [3]:
# Two firm algorithm markets
with open(ppj("OUT_DATA", f"grid_2_agents.pickle"), "rb") as f:
    all_output_grids_2_agents = pickle.load(f)
all_prices_2_agents_grid = np.array(all_output_grids_2_agents['avg_price'])
with open(ppj("OUT_DATA", "super_star_avg_prices_2_agents.pickle"), "rb") as f:
    super_star_avg_prices_2_agents = pickle.load(f)

# Three firm algorithm markets
with open(ppj("OUT_DATA", f"grid_3_agents.pickle"), "rb") as f:
    all_output_grids_3_agents = pickle.load(f)
all_prices_3_agents_grid = np.array(all_output_grids_3_agents['avg_price'])
with open(ppj("OUT_DATA", "super_star_avg_prices_3_agents.pickle"), "rb") as f:
    super_star_avg_prices_3_agents = pickle.load(f)


In [4]:
# we want to transform those to pandas dataframes as it will be easier to deal with them once we go over to plotting

In [5]:
with open(ppj("OUT_DATA", "data_group_level.pickle"), "rb") as f:
    data_group_level = pickle.load(f)

## Next up we use mann whitney u tests to compare the respective samples.

In [6]:
from scipy.stats import mannwhitneyu, ttest_1samp

### We start by comparing the market prices within a respective market size.

Two firm market:

In [7]:
print("Two firm market. Algo (super star) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                                        (data_group_level['treatment'] == '2H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = mannwhitneyu(
        subset_sg['winning_price'],
        super_star_avg_prices_2_agents,
        use_continuity=False,
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (super star) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.000
Super game: 3. P-value: 0.000


In [8]:
print("Two firm market. Algo (grid average) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                      (data_group_level['treatment'] == '2H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = ttest_1samp(
        subset_sg['winning_price'],
        all_prices_2_agents_grid.mean(),
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (grid average) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.005
Super game: 3. P-value: 0.034


In [9]:
print(f"Average price of the grid in two firm markets: {all_prices_2_agents_grid.mean():.3f}")

Average price of the grid in two firm markets: 3.511


Three firm market:


In [10]:
print("Three firm market. Algo (super star) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                     (data_group_level['treatment'] == '3H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = mannwhitneyu(
        subset_sg['winning_price'],
        super_star_avg_prices_3_agents,
        use_continuity=False,
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Three firm market. Algo (super star) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.008
Super game: 3. P-value: 0.980


In [11]:
print("Two firm market. Algo (grid average) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                      (data_group_level['treatment'] == '3H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = ttest_1samp(
        subset_sg['winning_price'],
        all_prices_3_agents_grid.mean(),
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (grid average) v human 
Super game: 1. P-value: 0.991
Super game: 2. P-value: 0.339
Super game: 3. P-value: 0.044


In [12]:
print(f"Average price of the grid in three firm markets: {all_prices_3_agents_grid.mean():.3f}")

Average price of the grid in three firm markets: 1.574


There exist no statistically significant differences between the prices in the three firm markets with the super star algorithms and humans. Interestingly, the average algorithm from the grid search is *less* collusive than the human market with this difference being statistically significant at $p<0.01$.

### Next, we compare the outcomes between market sizes.

In [14]:
mannwhitneyu(
    super_star_avg_prices_3_agents,
    super_star_avg_prices_2_agents,
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.0

In [15]:
mannwhitneyu(
    entire_price_grid_avg_prices_2_agents,
    entire_price_grid_avg_prices_3_agents,
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.0

Two firm algortihmic markets lead to hire markets prices compared to the case where three algorithms populate the same market ($p<0.01$).

In [16]:
a = data_humans.loc[data_humans['treatment'] == '3H0A']['winning_price'].values

In [17]:
mannwhitneyu(
    data_humans.loc[data_humans['treatment'] == '3H0A']['collusive'],
    data_humans.loc[data_humans['treatment'] == '2H0A']['collusive'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.46394035370574516

While market prices are higher in two firm human markets as suggested by the literature on human collusion, those difference are not statistically significant. This can also be confirmed by the means of regression anaylsis which you find in the appendix.

Note however that this is only the case in the last super game:



In [18]:
data_humans_first_super_game = data_group_level.loc[(data_group_level['super_game'] == 1) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()
data_humans_second_super_game = data_group_level.loc[(data_group_level['super_game'] == 2) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()
data_humans_third_super_game = data_group_level.loc[(data_group_level['super_game'] == 3) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


In [19]:
data_humans_all_super_game = data_group_level.loc[
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


In [20]:
mannwhitneyu(
    data_humans_first_super_game.loc[data_humans_first_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_first_super_game.loc[data_humans_first_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.04530386472278079

In [21]:
mannwhitneyu(
    data_humans_second_super_game.loc[data_humans_second_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_second_super_game.loc[data_humans_second_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.050961936967763306

In [24]:
mannwhitneyu(
    data_humans_third_super_game.loc[data_humans_third_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_third_super_game.loc[data_humans_third_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.28275614744399247

In [22]:
mannwhitneyu(
    data_humans_all_super_game.loc[data_humans_all_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_all_super_game.loc[data_humans_all_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.04042397933690852

In [23]:
data_humans_all_super_game

,super_group_id_general,treatment,winning_price,collusive
0,session_02062021_1000_1,3H0A,1.327044,0.037736
1,session_02062021_1000_2,3H0A,2.207547,0.364780
2,session_02062021_1130_1,3H0A,1.238994,0.075472
3,session_07062021_1130_1,3H0A,1.874214,0.251572
4,session_07062021_1130_2,3H0A,1.666667,0.327044
5,session_08062021_1630_3H0A_1,3H0A,3.283019,0.729560
6,session_17062021_1600_1,3H0A,1.452830,0.157233
7,session_20052021_1000_1,2H0A,2.622642,0.572327
8,session_20052021_1000_2,2H0A,2.767296,0.534591
9,session_21052021_1000_1,2H0A,3.421384,0.748428


This shows the importance of learning for humans, which appears to be more pronounce in larger and therefore strategically more complex markets.